inspiriert durch https://github.com/Trusted-AI/adversarial-robustness-toolbox/blob/main/examples/get_started_pytorch.py

In [1]:
from art.attacks.evasion import GRAPHITEWhiteboxPyTorch
from art.estimators.classification import PyTorchClassifier

from torchvision.transforms import ToTensor, Resize, Compose
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from pytorch_ood.utils import ToRGB
from gtsrb import GTSRB
from pathlib import Path
from PIL import Image
import numpy as np

/opt/anaconda3/envs/mls/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/anaconda3/envs/mls/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [4]:
trans = Compose([
            ToRGB(),
            ToTensor(),
            Resize((64, 64), antialias=True)
        ])


batch_size = 3
test_data = GTSRB(root=".", train=False, transforms=trans)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=2)

resnet18_model = torch.load("64x64/label-net-resnet18-64.pt", map_location="cpu", weights_only=False)
wideresnet40_model = torch.load("64x64/label-net-wrn40-64.pt", map_location="cpu", weights_only=False)

resnet18_model.eval()
wideresnet40_model.eval()


digital_imgs = Path("digital_imgs_1200")
images = []
to_tensor = ToTensor()
for img in sorted(digital_imgs.glob("*.png")):
    img = Image.open(img)
    img = img.resize((64, 64))
    img_t = to_tensor(img)
    images.append(img_t)

images = torch.stack(images)
labels = [16, 1, 38, 11, 33, 18, 9, 13, 34, 2, 7, 13, 26, 15, 28, 3]
labels = np.array(labels)

images.size()

torch.Size([16, 3, 64, 64])

testing on original images

In [5]:
with torch.no_grad():
    preds = wideresnet40_model(images).argmax(1)

accuracy = (preds == torch.tensor(labels)).float().mean().item()
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 1.00


In [9]:
# vorher mischen, sonst bleibt die Reihenfolge
perm = torch.randperm(len(images))
images = images[perm]
labels = labels[perm.numpy()]

# Splits
train_imgs, train_labels = images[:8], labels[:8]
test_imgs,  test_labels  = images[8:], labels[8:]


model = resnet18_model

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

classifier = PyTorchClassifier(
    model=model,
    clip_values=(0.0, 1.0),
    loss=criterion,
    optimizer=optimizer,
    input_shape=(3, 64, 64),
    nb_classes=43,
)

#classifier.fit(train_imgs, train_labels, batch_size=batch_size, nb_epochs=3)

predictions = classifier.predict(test_imgs)
pred_classes = np.argmax(predictions, axis=1)
accuracy = np.mean(pred_classes == test_labels)
print(f"Accuracy on val set: {accuracy * 100:.2f}%")

Accuracy on val set: 100.00%


In [8]:
x_test_np = test_imgs.detach().cpu().numpy()
y_test_np = test_labels.astype(np.int64)

attack = GRAPHITEWhiteboxPyTorch(classifier=classifier, net_size=(64, 64), num_xforms=100)
x_test_adv = attack.generate(x=x_test_np, y=y_test_np)
predictions = classifier.predict(x_test_adv)
accuracy_test = np.mean(pred_classes == test_labels)
print(f"Accuracy on test set: {accuracy_test * 100:.2f}%")


KeyboardInterrupt: 